In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
%matplotlib inline
#import evolution
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

INFO: [2017-10-04 10:34:01,271 - rospy.topics] topicmanager initialized


INFO: [2017-10-04 10:34:01,422 - Configuration] Loading configuration file config.json
INFO: [2017-10-04 10:34:01,425 - Configuration] Using user specified environment: local
INFO: [2017-10-04 10:34:01,469 - VirtualCoach] Ready.


## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [2]:
def fitness_function(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
    fitness = 0
    for i in range(len(left_wheel)):
        if left_wheel[i] >= 0 and right_wheel[i] >= 0:
                   fitness += (left_wheel[i] + right_wheel[i])
    return fitness/float(2*len(left_wheel))

def get_top_performers(population, fitness_log, num_performers=15):
    """
    Extract the indices of the top individuals from the fitness log.
    
    :param fitness_log: fitness function scores for all individuals in a population
    :param num_performers: number for top performers to look for. Default value is
                           15, which corresponds to a truncation threshold of 25% in
                           this experiment.
    """
    top_performers = []
    for i in range(num_performers):
        max_index = np.argmax(fitness_log)
        print max_index
        top_performers.append(population[max_index])
        fitness_log[max_index] = -1

    return top_performers

def plot_trajectory(trajectory):
    plt.figure()
    plt.gca()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.ylim(-3, 3)
    plt.xlim(-3.9, 3.9)
    x_axis = [x[0] for x in trajectory[2:]]
    y_axis = [y[1] for y in trajectory[2:]]
    plt.plot([float(x) for x in x_axis], [float(y) for y in y_axis])

def plot_wheel_speeds(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[1:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[1:]])
    plt.plot(range(len(left_wheel)), left_wheel, 'b')
    plt.plot(range(len(right_wheel)), right_wheel, 'r')

Evolutionary Algorithm helper functions

In [3]:
def one_point_crossover(parent1, parent2):
        parent1 = parent1.reshape(290)
        parent2 = parent2.reshape(290)
        child1 = np.zeros(290, dtype=int)
        child2 = np.zeros(290, dtype=int)
        point = np.random.randint(len(parent1))
        for i in range(point):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        for i in range(point, 290):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        child1 = child1.reshape(10, 29)
        child2 = child2.reshape(10, 29)
        return child1, child2
    
def bit_mutation(population):
        for individual in population:
            individual = individual.reshape(290)
            for i in range(290):
                if np.random.rand() < 0.05:
                    individual[i] = 0 if individual[i] else 1
        return population
    
def get_unique_pairs(population):
    pairs = []
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            pairs.append((i, j))
    return pairs
            
def evolve_new_generation(top_performers):
    population = []
    for i in range(len(top_performers)):
        for j in range(4):
            population.append(top_performers[i])
    pairs = get_unique_pairs(population)
    for i in pairs:
        if np.random.rand() < 0.1:
            parent1 = population[i[0]]
            parent2 = population[i[1] - 1]
            child1, child2 = one_point_crossover(parent1, parent2)
            population[i[0]] = child1
            population[i[1]] = child2

    population = bit_mutation(population)
    rand = np.random.randint(len(population))
    population[rand] = top_performers[0]
    return population

## The Brain
The PyNN script that creates the neural network stored as a string. A new binary genetic string that encodes the connections between neurons is passed on each run.

In [4]:
brain = 
"""
from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)

def create_brain():

    dna = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

    NEURONPARAMS = {'v_rest': -60.0,
                    'cm': 4.0,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -12.5,
                    'v_reset': -60.0}

    SYNAPSE_PARAMS = {"weight": 1.0,
                      "delay": 2.0}

    population = sim.Population(10, sim.IF_cond_alpha())
    population[0:10].set(**BRAINPARAMS)


    # Connect neurons
    CIRCUIT = population

    SYN = sim.StaticSynapse(**SYNAPSE_PARAMS)

    row_counter=0
    for row in dna:
    	logger.info(row)
        n = np.array(row)
        
        if n[0]==0:
            r_type = 'inhibitory'
        else:
            r_type = 'excitatory'
            
        for i in range(19,29):
            if n[i]==1:
                sim.Projection(presynaptic_population=CIRCUIT[18+row_counter:19+row_counter], postsynaptic_population=CIRCUIT[i-1:i], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        row_counter+=1

    sim.initialize(population, v=population.get('v_rest'))

    logger.debug("Circuit description: " + str(population.describe()))

    return population


circuit = create_brain()
"""

In [3]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

In [ ]:
sensor2brain = 
"""
import numpy as np
import cv2

@nrp.MapRobotSubscriber("camera", Topic('/husky/camera', sensor_msgs.msg.Image))

dna = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

connectors = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

row_counter=0
for row in dna:
    for i in range(1,19):
        if row[i]==1:
            name = "scon" + str(i) + "to" + str(row_counter)
            @nrp.MapSpikeSource(name, nrp.brain.brain[rowcounter], nrp.poisson)
            connectors[i-1].append(eval(name))
            clientLogger.info(name)
    row_counter+=1
    
connectors = np.array(connectors)
    
@nrp.MapVariable("ideal_wheel_speed", global_key="ideal_wheel_speed", initial_value=[0.0,0.0], scope=nrp.GLOBAL)
@nrp.MapVariable("real_wheel_speed", global_key="real_wheel_speed", initial_value=[0.0,0.0], scope=nrp.GLOBAL)

@nrp.Robot2Neuron()

def Sensor2Brain(t, ideal_wheel_speed, real_wheel_speed, camera, connectors:

    connections = np.array(connectors)
    bridge = CvBridge()

    if not isinstance(camera.value, type(None)):

        # Get an OpenCV image converted to a 8-bit greyscale image and to a binary black and white image.
        (thresh, im_bw) = cv2.threshold(bridge.imgmsg_to_cv2(camera.value, "mono8"), 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        # Fold the image to get a 16 values used to set the firing rates. 4 pixels are collapsed into one value with a weighted sum (center pixels have double weights)
        for n in range(16):
            r = (im_bw.item(n)/6.0) + (im_bw.item(n+1)/3.0) + (im_bw.item(n+2)/3.0) + (im_bw.item(n+3)/6.0)
            for x in connections[n]
                x.rate = r


    iws = ideal_wheel_speed.value
    rws = real_wheel_speed.value
    for x in connections[16]
        x.rate = np.absolute(100.0*iws[0]-rws[0])
    for y in connections[17]
        y.rate = np.absolute(100.0*iws[1]-rws[1])

## Run Experiment

In [6]:
class FloreanoExperiment(object):
    
    def __init__(self, population, generations):
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')
        
    def on_status(self, status):
        self.last_status[0] = status
                 
    def save_simulation_data(self, trial):
        self.sim_data[trial].append([])
        wheel_speeds = self.sim.get_csv_data('wheel_speeds.csv')
        left_wheel = ([float(t[1]) for t in wheel_speeds[2:]])
        right_wheel = ([float(t[2]) for t in wheel_speeds[2:]])
        trajectory = self.sim.get_csv_data('robot_position.csv')
        fitness = fitness_function(wheel_speeds)
        self.sim_data[trial][-1] = {
            'fitness': fitness,
            'wheel_speeds': wheel_speeds,
            'left_wheel': left_wheel,
            'right_wheel': right_wheel,
            'trajectory': trajectory
        }

    def run_experiment(self):
        try:
            self.sim = vc.launch_experiment('floreano')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            self.sim_data.append([])
            for j in range(len(self.population)):
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(brain % genetic_string)
                self.sim.edit_transfer_function(sensor2brain % genetic_string)
                self.sim.add_transfer_function(display_trial_tf % "Generation {}, Population {}".format(i, j) )
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(1000, lambda x: x['simulationTime'] == 40)
                self.sim.pause()
                self.save_simulation_data(i)
                self.sim.reset('full')
                self.wait_condition(100, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            self.fitness_log = [result['fitness'] for result in floreano_experiment.sim_data[i]]
            self.top_performers = get_top_performers(self.population, list(self.fitness_log))
            self.population = evolve_new_generation(self.top_performers)

In [ ]:
population = np.random.randint(2, size=(60, 10, 29)) # random population of 30 binary genetic strings
floreano_experiment = FloreanoExperiment(population, 16)
floreano_experiment.run_experiment()

INFO: [2017-10-04 10:34:11,552 - VirtualCoach] Preparing to launch floreano.
INFO: [2017-10-04 10:34:11,552 - VirtualCoach] Retrieving list of experiments.
INFO: [2017-10-04 10:34:11,575 - Simulation] Attempting to launch floreano on localhost.
INFO: [2017-10-04 10:34:21,422 - Simulation (floreano - localhost #0)] Simulation Successfully Created.
INFO: [2017-10-04 10:34:21,432 - Simulation (floreano - localhost #0)] Ready.
INFO: [2017-10-04 10:34:21,432 - Simulation (floreano - localhost #0)] Status callback registered.
INFO: [2017-10-04 10:34:21,433 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:34:21,439 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2017-10-04 10:34:21,440 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:34:21,456 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:34:21,469 - Simulation (floreano - localhost #0)] Attempti

INFO: [2017-10-04 10:35:56,672 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:35:56,676 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:35:57,031 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2017-10-04 10:35:57,032 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2017-10-04 10:35:57,038 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2017-10-04 10:35:57,048 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2017-10-04 10:35:57,049 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2017-10-04 10:35:57,059 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2017-10-04 10:36:37,847 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2017-10-04 10:36:37,864 - Simulation 

INFO: [2017-10-04 10:38:09,071 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2017-10-04 10:38:09,072 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 10:38:09,118 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 10:38:09,145 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 10:38:09,208 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 10:38:09,231 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2017-10-04 10:38:09,237 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2017-10-04 10:38:09,238 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2017-10-04 10:38:09,863 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2017-10-04 10:38:09,864 - Simulation 

INFO: [2017-10-04 10:39:45,959 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2017-10-04 10:39:45,969 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2017-10-04 10:39:45,970 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2017-10-04 10:39:46,375 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2017-10-04 10:39:46,376 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2017-10-04 10:39:46,734 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2017-10-04 10:39:46,735 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2017-10-04 10:39:46,946 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2017-10-04 10:39:46,947 - Simulation (floreano - 

INFO: [2017-10-04 10:41:19,578 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2017-10-04 10:41:19,797 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2017-10-04 10:41:20,173 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-10-04 10:41:20,675 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:41:20,686 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2017-10-04 10:41:20,687 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:41:20,692 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:41:20,701 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 10:41:20,705 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04

...

INFO: [2017-10-04 19:57:28,751 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2017-10-04 19:57:29,278 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-10-04 19:57:30,280 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 19:57:30,289 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2017-10-04 19:57:30,289 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 19:57:30,300 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 19:57:30,308 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 19:57:30,316 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 19:57:30,804 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2017-10-04 19:57:30,805 

INFO: [2017-10-04 20:02:24,796 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2017-10-04 20:02:24,806 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2017-10-04 20:02:24,823 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2017-10-04 20:02:24,824 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2017-10-04 20:02:24,829 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2017-10-04 20:04:34,231 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2017-10-04 20:04:34,235 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2017-10-04 20:04:34,236 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 20:04:34,255 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 2

INFO: [2017-10-04 20:08:29,176 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 20:08:29,191 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2017-10-04 20:08:29,221 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2017-10-04 20:08:29,230 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2017-10-04 20:08:29,232 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2017-10-04 20:08:30,073 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2017-10-04 20:08:30,074 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2017-10-04 20:08:30,084 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2017-10-04 20:08:30,085 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO:

INFO: [2017-10-04 20:12:24,593 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2017-10-04 20:12:24,594 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2017-10-04 20:12:24,990 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2017-10-04 20:12:24,991 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2017-10-04 20:12:25,197 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2017-10-04 20:12:25,198 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2017-10-04 20:12:25,422 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2017-10-04 20:12:25,834 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
INFO: [2017-10-04 20:12:26,586 

INFO: [2017-10-04 20:16:21,621 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 20:16:21,630 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2017-10-04 20:16:21,631 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 20:16:21,639 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 20:16:21,644 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 20:16:21,650 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2017-10-04 20:16:22,068 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2017-10-04 20:16:22,069 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2017-10-04 20:16:22,081 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2017-10-04 20:16:22,095 - Simulation (floreano - localhost #0)] Transfer Functi

In [ ]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][2:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][2:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)
